This notebook has been created to extract the PE scores for each protein in the sus scrofa proteome database (it can be edited to allow the input of any FASTA file), group the proteins to their respective genes and output the following dataframes:

1. A dataframe which has unique gene names and the number of PE scores (grouped by 1, 2, 3, 4, 5) belonging to each gene (found in the output folder as no_sum_sus_scrofa_PE.csv)
2. A processed dataframe taken from (1) to only pass gene names where it has at least two unique nonzero PE scores and has an additional column adding the PE scores across the row (found in the output folder as sus_scrofa_PE_sum_table.csv)

In [ ]:
# Connect notebook to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Navigate to the shared drive folder
%cd /content/drive/'My Drive'/'Pig Proteomics Summer Project'
!pwd
!ls

/content/drive/.shortcut-targets-by-id/1H3M9nQYJ9K9M22jDv-wcvZ0azPz9lhsr/Pig Proteomics Summer Project
/content/drive/.shortcut-targets-by-id/1H3M9nQYJ9K9M22jDv-wcvZ0azPz9lhsr/Pig Proteomics Summer Project
'2021-06-21 Introduction to Proteomes.ipynb'
'2021-06-28 Reactome and Uniprot.ipynb'
'2021-06-29 Identifying characteristics of an unreliable protein.gslides'
 data
 extract_106.ipynb
 gene_PE.ipynb
 histogram.ipynb
 output
 pe_scores.ipynb
 scratch
'Summer Project.gdoc'
 violin.ipynb
'Week 1 Introduction slides.gslides'
'Week 3 Findings.gslides'


In [ ]:
!pip install fastaparser

# **FUNCTIONS**

In [ ]:
import fastaparser
import pandas as pd
import re

def parse_fasta_file(file_name):
    """
    this function parses a FASTA database and returns it as a mapping from the
    header of the protein to the amino acid sequence it corresponds to
    mapping: {header: sequence}
    """
    out_dict = {}
    fastaRaw = [] #uses quick parsing method
    with open(file_name) as fasta_file:
        parserQuick = fastaparser.Reader(fasta_file, parse_method = 'quick')
        
        for seq in parserQuick:
            fastaRaw.append(seq)
            
    headerList = []
    seqList = []
    
    for k in fastaRaw:
        headerList.append(k.header)
        seqList.append(k.sequence)
    
    if len(headerList) != len(seqList):
        raise ValueError("Number of headers =/= number of sequences")
    
    for i in range(len(headerList)):
        out_dict[headerList[i]] = seqList[i]
    
    return out_dict


def fasta_to_table(file_name):
    """
    creates function which accepts a FASTA file and returns dataframe in this format:
    SP or TR | Accession | Organism | Gene Name | Length
    --------------------------------------------------------
    if it is swissprot or trembl, accession, organism, its gene name, and length
    """
    header = ['sptr', 'Accession', 'Organism', 'Gene name', 'Length']
    sptr = []
    accession = []
    organism = []
    gene_name = []
    length = []
    fastaRaw = []
    PE_scores = []
    indice1 = 0 #indice for 'OS='
    indice2 = 0 #indice for 'OX='
    indice3 = 0 #indice for 'GN='
    indice4 = 0 #indice for 'PE='
    
    out_dict = parse_fasta_file(file_name)
    
    for key in out_dict.keys():
        indice1 = True #indice for 'OS='
        indice2 = True #indice for 'OX='
        indice3 = True #indice for 'GN='
        indice4 = True #indice for 'PE='
        indice5 = True #indice for 'SV='
        
        indice = []

        t1 = key[1:3] #sp or tr

        for j in range(len(key)):
            t_find = key[j:j+1]
            if t_find == '|':
                indice.append(j)
            else:
                pass
        
        t2 = key[indice[0]+1:indice[1]] #accession
        
        for k in range(len(key)):
            dummyText = key[k:k+3]
            if dummyText == 'OS=':
              indice1 = k
            elif dummyText == 'OX=':
              indice2 = k
            elif dummyText == 'GN=':
              indice3 = k
            elif dummyText == 'PE=':
              indice4 = k
            elif dummyText == 'SV=':
              indice5 = k
            else:
                pass
        
        if indice3 == True:
          GNName = 'No Name'

        else:
          GNName = key[indice3+3:indice4]
          
        OSName = key[indice1+3:indice2]
        lengthCalc = len(out_dict[key])
        PE = int(key[indice4+3:indice5])
        
        sptr.append(t1)
        accession.append(t2)
        organism.append(OSName)
        gene_name.append(GNName)
        length.append(lengthCalc)
        PE_scores.append(PE)
        
    data = {'sptr' : sptr, 'accession' : accession, 'organism' : organism, 'Gene Name' : gene_name,
           'length' : length, 'PE Score' : PE_scores}
    df = pd.DataFrame(data)
    
    ''' how alex would have done it
        sub_table = fasta_table[fasta_table['GN'] == i]
        pig_list = set(sub_table[sub_table['organism'] == 'Sus scrofa']['accession])
        '''
    return df

def PE_score_table(fasta_table):
  """
  Gene Name | #PE = 1 | #PE = 2 | #PE = 3 | #PE = 4 | #PE = 5
  -------------------------------------------------------------

  headers(fasta_table) = ['sptr','accession','organism','Gene Name','Length','PE Score']
  """
  fasta_table['Gene Name'] = [x.upper() for x in list(fasta_table['Gene Name'])]
  unique_gene_names = list(set(list(fasta_table['Gene Name'])))

  #gene_to_PE = {}
  PE_1 = [] #number of proteins in gene name with PE = 1
  PE_2 = [] #number of proteins in gene name with PE = 2
  PE_3 = [] #number of proteins in gene name with PE = 3
  PE_4 = [] #number of proteins in gene name with PE = 4
  PE_5 = [] #number of proteins in gene name with PE = 5

  for i in unique_gene_names:
    PE_1_num = list(fasta_table.loc[fasta_table['Gene Name'] == i]['PE Score']).count(1)
    PE_2_num = list(fasta_table.loc[fasta_table['Gene Name'] == i]['PE Score']).count(2)
    PE_3_num = list(fasta_table.loc[fasta_table['Gene Name'] == i]['PE Score']).count(3)
    PE_4_num = list(fasta_table.loc[fasta_table['Gene Name'] == i]['PE Score']).count(4)
    PE_5_num = list(fasta_table.loc[fasta_table['Gene Name'] == i]['PE Score']).count(5)

    PE_1.append(PE_1_num)
    PE_2.append(PE_2_num)
    PE_3.append(PE_3_num)
    PE_4.append(PE_4_num)
    PE_5.append(PE_5_num)
  
  d = {
      'Gene Name' : unique_gene_names,
      '#PE = 1' : PE_1,
      '#PE = 2' : PE_2,
      '#PE = 3' : PE_3,
      '#PE = 4' : PE_4,
      '#PE = 5' : PE_5
  }

  gene_to_PE_score = pd.DataFrame(data = d)

  return gene_to_PE_score


In [ ]:
%cd data

/content/drive/.shortcut-targets-by-id/1H3M9nQYJ9K9M22jDv-wcvZ0azPz9lhsr/Pig Proteomics Summer Project/data


# **SUS SCROFA**

In [ ]:
pig_proteome_file = 'sus_scrofa_uniprot-proteome_UP000008227.fasta'
pig_proteome_table = fasta_to_table(pig_proteome_file)
pig_proteome_table

,sptr,accession,organism,Gene Name,length,PE Score
0,sp,Q52NJ3,Sus scrofa,SAR1A,198,2
1,sp,Q6PKU1,Sus scrofa,SPI1,270,2
2,sp,P80928,Sus scrofa,MIF,115,1
3,sp,D5K8A2,Sus scrofa,SPATA18,560,2
4,sp,Q2HYU2,Sus scrofa,PFKM,780,2
...,...,...,...,...,...,...
49787,tr,A0A287B7R5,Sus scrofa,MEX3B,576,4
49788,tr,I3LQB3,Sus scrofa,ATP6V1H,465,3
49789,tr,K7GNX3,Sus scrofa,SLC25A14,344,3
49790,tr,A0A287AKI0,Sus scrofa,BCLAF1,921,1


In [ ]:
pig_proteome_table.to_csv("pig_proteome_table_PE.csv")

In [ ]:
pig_all_file = 'sus_scrofa_all.fasta'
pig_all_table = fasta_to_table(pig_all_file)
pig_all_table

,sptr,accession,organism,Gene Name,length,PE Score
0,sp,Q9TV69,Sus scrofa,DHDH,335,1
1,sp,P27594,Sus scrofa,MX1,663,2
2,sp,P00355,Sus scrofa,GAPDH,333,1
3,sp,Q5I2M3,Sus scrofa,TLR9,1030,2
4,sp,Q95JC9,Sus scrofa,No Name,676,1
...,...,...,...,...,...,...
120921,sp,Q8SQ26,Sus scrofa,CRYL1,322,2
120922,sp,Q6Q7J2,Sus scrofa,GDI2,445,2
120923,sp,F1S5L4,Sus scrofa,GPAM,826,2
120924,sp,P50132,Sus scrofa,GPR4,363,3


In [ ]:
pig_all_table.to_csv("pig_all_table_PE.csv")

In [ ]:
pig_proteome_table[pig_proteome_table['Gene Name'] == 'KCTD4 ']

,sptr,accession,organism,Gene Name,length,PE Score
2826,tr,I3LNK6,Sus scrofa,KCTD4,259,4


In [ ]:
print("Total #PE Score 1: ", len(list(pig_proteome_table.loc[pig_proteome_table['PE Score'] == 1, 'Gene Name'])))
print("Total #PE Score 2: ", len(list(pig_proteome_table.loc[pig_proteome_table['PE Score'] == 2, 'Gene Name'])))
print("Total #PE Score 3: ", len(list(pig_proteome_table.loc[pig_proteome_table['PE Score'] == 3, 'Gene Name'])))
print("Total #PE Score 4: ", len(list(pig_proteome_table.loc[pig_proteome_table['PE Score'] == 4, 'Gene Name'])))
print("Total #PE Score 5: ", len(list(pig_proteome_table.loc[pig_proteome_table['PE Score'] == 5, 'Gene Name'])))


Total #PE Score 1:  17008
Total #PE Score 2:  1281
Total #PE Score 3:  13041
Total #PE Score 4:  18462
Total #PE Score 5:  0


In [ ]:
sus_scrofa_PE_table = PE_score_table(pig_proteome_table)
sus_scrofa_PE_table

,Gene Name,#PE = 1,#PE = 2,#PE = 3,#PE = 4,#PE = 5
0,NANOS2,0,0,1,0,0
1,LRRK1,0,0,0,3,0
2,DAPK2,0,0,5,1,0
3,SLC26A10,0,0,1,0,0
4,FGF6,0,0,1,0,0
...,...,...,...,...,...,...
18016,GOLPH3,0,1,0,0,0
18017,LOC102159509,0,0,0,1,0
18018,TMEM160,0,0,0,1,0
18019,SNX11,0,0,2,0,0


In [ ]:
pig_PE_table_copy = sus_scrofa_PE_table.copy()

In [ ]:
PE_scores = [x[1:6] for x in pig_PE_table_copy.values.tolist()]
num_zero = [x.count(0) for x in PE_scores]
pig_PE_table_copy['Number of null PE'] = num_zero
#sum_PE = [sum(x) for x in PE_scores]
#sus_scrofa_PE_table['Sum of Proteins'] = sum_PE

In [ ]:
pig_PE_table_copy

,Gene Name,#PE = 1,#PE = 2,#PE = 3,#PE = 4,#PE = 5,Number of null PE
0,NANOS2,0,0,1,0,0,4
1,LRRK1,0,0,0,3,0,4
2,DAPK2,0,0,5,1,0,3
3,SLC26A10,0,0,1,0,0,4
4,FGF6,0,0,1,0,0,4
...,...,...,...,...,...,...,...
18016,GOLPH3,0,1,0,0,0,4
18017,LOC102159509,0,0,0,1,0,4
18018,TMEM160,0,0,0,1,0,4
18019,SNX11,0,0,2,0,0,4


In [ ]:
df_new = pig_PE_table_copy.drop(pig_PE_table_copy[(pig_PE_table_copy['Number of null PE'] == 4)].index)

In [ ]:
del df_new['Number of null PE']

In [ ]:
sum_PE = [sum(x) for x in [i[1:6] for i in df_new.values.tolist()]]
df_new['Sum of Proteins'] = sum_PE
df_new

,Gene Name,#PE = 1,#PE = 2,#PE = 3,#PE = 4,#PE = 5,Sum of Proteins
2,DAPK2,0,0,5,1,0,6
18,NCOA7,0,0,3,1,0,4
23,UPK1A,0,1,0,2,0,3
44,FFAR1,0,1,1,0,0,2
54,OPTN,1,0,0,4,0,5
...,...,...,...,...,...,...,...
17989,SLC44A2,4,0,2,0,0,6
17992,TBCE,6,0,1,0,0,7
18009,VIPR1,0,2,4,0,0,6
18011,KDM5A,0,0,4,1,0,5


In [ ]:
S1 = sum(list(sus_scrofa_PE_table['#PE = 1']))
S1

17008

In [ ]:
S2 = sum(list(sus_scrofa_PE_table['#PE = 2']))
S2

1281

In [ ]:
S3 = sum(list(sus_scrofa_PE_table['#PE = 3']))
S3

13041

In [ ]:
S4 = sum(list(sus_scrofa_PE_table['#PE = 4']))
S4

18462

In [ ]:
S1 + S2 + S3 + S4

49792

In [ ]:
df_new.to_csv("sus_scrofa_PE_sum_table.csv")

In [ ]:
sus_scrofa_PE_table.to_csv("no_sum_sus_scrofa_PE.csv")

# **HOMO SAPIEN**

In [ ]:
human_proteome_file = 'Homo_sapiens_uniprot-proteome_UP000005640.fasta'
human_proteome_table = fasta_to_table(human_proteome_file)
human_proteome_table

,sptr,accession,organism,Gene Name,length,PE Score
0,sp,Q00266,Homo sapiens,MAT1A,395,1
1,sp,Q8NB16,Homo sapiens,MLKL,471,1
2,sp,O94851,Homo sapiens,MICAL2,1124,1
3,sp,Q8TDZ2,Homo sapiens,MICAL1,1067,1
4,sp,Q9NPJ6,Homo sapiens,MED4,270,1
...,...,...,...,...,...,...
78115,tr,A0A087WVB8,Homo sapiens,CNBD2,181,1
78116,tr,H0Y876,Homo sapiens,CCDC136,548,1
78117,tr,A0A3B3IU88,Homo sapiens,No Name,217,4
78118,tr,A0A1W2PR38,Homo sapiens,CSAG2,78,4


In [ ]:
human_proteome_table.to_csv("human_proteome_table_PE.csv")

In [ ]:
human_all_file = 'homo_sapien_all.fasta'
human_all_table = fasta_to_table(human_all_file)
human_all_table

,sptr,accession,organism,Gene Name,length,PE Score
0,sp,P04439,Homo sapiens,HLA-A,365,1
1,sp,P01911,Homo sapiens,HLA-DRB1,266,1
2,sp,P01889,Homo sapiens,HLA-B,362,1
3,sp,P31689,Homo sapiens,DNAJA1,397,1
4,sp,P08246,Homo sapiens,ELANE,267,1
...,...,...,...,...,...,...
202155,sp,Q9P2T1,Homo sapiens,GMPR2,348,1
202156,sp,Q8IUX8,Homo sapiens,EGFL6,553,1
202157,sp,O15372,Homo sapiens,EIF3H,352,1
202158,sp,P54852,Homo sapiens,EMP3,163,1


Converting PE tables to CSV

In [ ]:
human_all_table.to_csv("human_all_table_PE.csv")

# **MUS MUSCULUS**

In [ ]:
mouse_proteome_file = 'mus_musculus_uniprot-proteome_UP000000589.fasta'
mouse_proteome_table = fasta_to_table(mouse_proteome_file)
mouse_proteome_table

,sptr,accession,organism,Gene Name,length,PE Score
0,sp,Q9D2V8,Mus musculus,Mfsd10,456,1
1,sp,Q8VHK5,Mus musculus,Mlc1,382,1
2,sp,Q5SW45,Mus musculus,Mks1,561,1
3,sp,Q3UQI9,Mus musculus,Mindy4,744,1
4,sp,Q9D273,Mus musculus,Mmab,237,1
...,...,...,...,...,...,...
55361,tr,D3Z0K8,Mus musculus,Txnrd2,466,1
55362,tr,A0A1L1STA1,Mus musculus,Csk,139,1
55363,tr,A0A0G2JF94,Mus musculus,Trav14-1,121,1
55364,tr,G5E8Y0,Mus musculus,Slc8a1,970,1


In [ ]:
mouse_proteome_table.to_csv("mouse_proteome_table_PE.csv")

In [ ]:
mouse_all_file = 'mus_musculus_all.fasta'
mouse_all_table = fasta_to_table(mouse_all_file)
mouse_all_table

,sptr,accession,organism,Gene Name,length,PE Score
0,sp,P02762,Mus musculus,Mup6,180,1
1,sp,Q91ZJ0,Mus musculus,Mus81,551,1
2,tr,Q6ZQL8,Mus musculus,Gbp6,611,2
3,sp,Q8R4K8,Mus musculus,Pappa,1624,2
4,sp,P02089,Mus musculus,Hbb-b2,147,1
...,...,...,...,...,...,...
86516,sp,Q9Z0F1,Mus musculus,Gnas,257,2
86517,sp,Q9DCZ1,Mus musculus,Gmpr,345,1
86518,sp,Q9EQQ3,Mus musculus,Gpr63,425,2
86519,sp,Q99L27,Mus musculus,Gmpr2,348,1


In [ ]:
mouse_all_table.to_csv("mouse_all_table_PE.csv")